# MBA FIAP Inteligência Artificial & Machine Learning

## Visão Computacional: Análise de Imagens Médicas

> Atenção: este notebook foi desenhado para funcionar no **Google Collab**.


## 1. Introdução

Uma determinada fintech focada em consumidores finais pessoa física constataou um grande número de fraudes em transações bancárias.

O setor de fraudes apontou que existem clientes que se queixaram de não contratar serviços específicos, como o crédito pessoal, e após isso transferir para outras contas desconhecidas.

Após análises pelas equipes de segurança, os protocolos de utilização da senha foram realizados em conformidade, ou seja, cada cliente autenticou com sua própria senha de maneira regular.

Em função disso, o banco precisa arcar com reembolsos e medidas de contenção para evitar processos judiciais, pois os clientes alegam terem sido invadidos por hackers ou algo parecido.

Uma das formas de solucionar ou minimizar este problema é com a utilização de outras formas de autenticação, sobretudo em operações críticas, como a obtenção de crédito pessoal.

Desta forma podemos implementar uma verificação de identidade com prova de vida (liveness), que utilize uma verificação e identificação facial.

Caso o cliente não seja autenticado, ele será atendido por uma esteira dedicada e as evidências da não identificação serão encaminhadas para a área de IA para validação dos parâmetros e limiares para aperfeiçoamento do modelo.

Será necessário construir:

* Detector de faces
* Identificação de faces (podendo ser um comparador entre um rosto de documento e outra da prova de vida)
* Detecção de vivacidade (liveness) para evitar que um fraudador utilize uma foto estática.


>Formas alternativas de prover a identificação e prova de vivacidade, além destas que foram solicitadas poderão ser submetidas.


<p align="center">
    <img src="https://github.com/michelpf/fiap-ml-visao-computacional-detector-liveness/blob/master/notebook/imagens/liveness.jpg?raw=1">
</p>

Imagem retirada do [Grunge](https://www.grunge.com/192826/company-testing-robocop-facial-recognition-software-with-us-police/).

## 2. Instruções

Este projeto final tem como objetivo explorar os conhecimentos adquiridos nas aulas práticas.

Iremos constuir uma forma de validar se uma determinada imagem foi ou não adulterada e se trata de uma produção fraudade.

Existem diversas formas de validar a vivacidade, e neste sentido conto com a criatividade de vocês dado que já dominam encontrar uma face numa imagem, aplicar marcos faciais e até mesmo construir uma rede neural convulacional.

A abordagem mais simples é pela construção de uma rede neural com imagens de fotos de rostos de outras fotos e fotos de rostos sem modificações. Tal classificador deverá classificar se dada imagem possui vivacidade ou não com uma pontuação de probabilidade.

Referências que abordam o tema para servir de inspiração:

1. [PyImageSearch](https://pyimagesearch.com/2019/03/11/liveness-detection-with-opencv/), Liveness detection with OpenCV;
2. [Kickertech](https://kickertech.com/face-liveness-detection-via-opencv-and-tensorflow/), Liveness detection via OpenCV and Tensorflow.
3. [Towards Data Science](https://towardsdatascience.com/real-time-face-liveness-detection-with-python-keras-and-opencv-c35dc70dafd3?gi=24f8e1b740f9), Real-time face liveness detection with Python, Keras and OpenCV.

Este projeto poderá ser feita por grupos de até 4 pessoas.
Caso este projeto seja substitutivo, deverá ser realizado por apenas uma pessoa.

| Nome dos Integrantes     | RM            | Turma |
| :----------------------- | :------------- | :-----: |
| Integrante 1             | RM 12345      | XIA |
| Integrante 2             | RM 12345      | XIA |
| Integrante 3             | RM 12345      | XIA |
| Integrante 4             | RM 12345      | XIA |

## 3. Abordagem e organização da solução do problema (2 pontos)

Como o grupo pretende deteccar a prova de vivacidade de uma determinada imagem? Quais os passos e os building blocks deste processo?

Gravaremos dois vídeos de cada integrante do grupo: um original e outro simulado. Usaremos esses vídeos para detectar rostos em cada frame e salvá-los em duas pastas: "real" e "falso". Essas imagens serão utilizadas para treinar um modelo de rede neural. Após o treinamento, o modelo será capaz de analisar características das imagens e determinar se são reais ou falsas. Por fim, a rede neural poderá ser usada com a câmera do computador ou com o upload de fotos para detectar a autenticidade das imagens.

**Resposta**:

## 4 Desenvolvimento da solução (5,5 pontos)

Detalhe o passo-a-passo do algoritmo de deteção de vivacidade.
Se optar pela construção e treinamento de um modelo de redes neurais convulucionais, apresente a arquitetura, prepare os dados de treinamento, realize o treinamento.

### 4.0 Importação de bibliotecas

In [29]:
import numpy as np
import cv2
import os
import pickle
import imutils
import time
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
from imutils.video import VideoStream

### 4.1 Organização de dados para treinamento de modelo de liveness (2 pontos)

In [11]:
# Define o valor mínimo de confiança para considerar uma detecção de rosto
confidence_value = 0.50

# Carrega o modelo de detecção de rostos do Caffe
net = cv2.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")

# Lista todos os vídeos no diretório "./videos/"
videos = os.listdir("./videos/")
for video in videos:
    # Ignora a pasta "processados"
    if(video == "processados"):
        break
    
    # Extrai o nome e o tipo do vídeo a partir do nome do arquivo
    infos = video.split(".")[0].split("_")
    nome = infos[0]
    tipo = infos[1]

    # Abre o vídeo para leitura
    vs = cv2.VideoCapture("./videos/" + video)
    saved_path = "./dataset/" + tipo
    read = 0
    saved = 0

    while True:
        # Lê um frame do vídeo
        (grabbed, frame) = vs.read()
        
        # Interrompe após ler 100 frames
        if read >= 100:
            break
        read += 1
        
        # Obtém as dimensões do frame e prepara o blob para a detecção de rostos
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))
        
        # Passa o blob pela rede e obtém as detecções de rostos
        net.setInput(blob)
        detections = net.forward()

        # Se houver detecções
        if len(detections) > 0:
            # Pega o índice da detecção com maior confiança
            i = np.argmax(detections[0, 0, :, 2])
            confidence = detections[0, 0, i, 2]

            # Se a confiança for maior que o valor mínimo definido
            if confidence > confidence_value:
                # Calcula as coordenadas da caixa delimitadora do rosto
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                
                # Extrai o rosto da imagem
                face = frame[startY:endY, startX:endX]
                
                # Define o caminho e salva a imagem do rosto no disco
                p = os.path.sep.join([saved_path, "{}_{}_{}.png".format(nome, tipo, saved)])
                cv2.imwrite(p, face)
                saved += 1
                print("[INFO] saved {} to disk".format(p))
    
    # Libera o vídeo e fecha todas as janelas do OpenCV
    vs.release()
cv2.destroyAllWindows()

[INFO] saved ./dataset/falso\bruno_falso_0.png to disk
[INFO] saved ./dataset/falso\bruno_falso_1.png to disk
[INFO] saved ./dataset/falso\bruno_falso_2.png to disk
[INFO] saved ./dataset/falso\bruno_falso_3.png to disk
[INFO] saved ./dataset/falso\bruno_falso_4.png to disk
[INFO] saved ./dataset/falso\bruno_falso_5.png to disk
[INFO] saved ./dataset/falso\bruno_falso_6.png to disk
[INFO] saved ./dataset/falso\bruno_falso_7.png to disk
[INFO] saved ./dataset/falso\bruno_falso_8.png to disk
[INFO] saved ./dataset/falso\bruno_falso_9.png to disk
[INFO] saved ./dataset/falso\bruno_falso_10.png to disk
[INFO] saved ./dataset/falso\bruno_falso_11.png to disk
[INFO] saved ./dataset/falso\bruno_falso_12.png to disk
[INFO] saved ./dataset/falso\bruno_falso_13.png to disk
[INFO] saved ./dataset/falso\bruno_falso_14.png to disk
[INFO] saved ./dataset/falso\bruno_falso_15.png to disk
[INFO] saved ./dataset/falso\bruno_falso_16.png to disk
[INFO] saved ./dataset/falso\bruno_falso_17.png to disk
[I

### 4.2 Treinamento de modelo de liveness (1,5 pontos)

In [12]:
def build(width, height, depth, classes):
	# Inicializa o modelo sequencial e a forma de entrada
	model = Sequential()
	inputShape = (height, width, depth)
	chanDim = -1
	
	# Ajusta a forma de entrada se o formato da imagem for "channels_first"
	if K.image_data_format() == "channels_first":
		inputShape = (depth, height, width)
		chanDim = 1
		
	# Primeira camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(16, (3, 3), padding="same", input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Segunda camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(16, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Primeira camada de pooling e dropout
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	
	# Terceira camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Quarta camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Segunda camada de pooling e dropout
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	
	# Camada de flatten para transformar os dados em um vetor 1D
	model.add(Flatten())
	
	# Camada densa com ativação ReLU e normalização em lote
	model.add(Dense(64))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	
	# Camada de saída com ativação softmax para classificação
	model.add(Dense(classes))
	model.add(Activation("softmax"))
	
	# Retorna o modelo construído
	return model

In [16]:
# Definindo o tamanho do batch e o número de épocas
BS = 8
EPOCHS = 50

print("[INFO] loading images...")

# Carrega as imagens do diretório "./dataset/"
print("[INFO] loading images...")
imagePaths = list(paths.list_images("./dataset/"))
data = []
labels = []

# Loop através dos caminhos das imagens
for imagePath in imagePaths:
	# Extrai o rótulo da imagem a partir do caminho do arquivo
	label = imagePath.split(os.path.sep)[-2]
	
	# Carrega a imagem, redimensiona para 32x32 pixels e adiciona aos dados
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32))
	data.append(image)
	
	# Converte o rótulo em "Falso" ou "Real"
	labels.append("Falso" if label == "./dataset/falso" else "Real")

# Converte os dados em um array numpy e normaliza os valores dos pixels para o intervalo [0, 1]
data = np.array(data, dtype="float") / 255.0

# Codifica os rótulos como inteiros e, em seguida, converte para vetores one-hot
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# Divide os dados em conjuntos de treinamento e teste
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# Inicializa o gerador de dados para aumentar os dados de treinamento
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                         height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

print("[INFO] compiling model...")

# Constrói o modelo usando a função definida anteriormente
model = build(width=32, height=32, depth=3, classes=len(le.classes_))

# Compila o modelo usando a função de perda "binary_crossentropy" e o otimizador "adam"
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print("[INFO] training network for {} epochs...".format(EPOCHS))

# Treina a rede neural utilizando os dados de treinamento aumentados e valida utilizando os dados de teste
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS), validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)


[INFO] loading images...
[INFO] loading images...
[INFO] compiling model...
[INFO] training network for 50 epochs...
Epoch 1/50
75/75 [==============================] - 3s 14ms/step - loss: 0.3675 - accuracy: 0.8667 - val_loss: 0.5182 - val_accuracy: 0.6050
Epoch 2/50
75/75 [==============================] - 1s 12ms/step - loss: 0.1825 - accuracy: 0.9367 - val_loss: 1.1705 - val_accuracy: 0.5350
Epoch 3/50
75/75 [==============================] - 1s 14ms/step - loss: 0.1284 - accuracy: 0.9600 - val_loss: 1.2045 - val_accuracy: 0.5100
Epoch 4/50
75/75 [==============================] - 1s 14ms/step - loss: 0.1357 - accuracy: 0.9517 - val_loss: 2.4940 - val_accuracy: 0.5100
Epoch 5/50
75/75 [==============================] - 1s 14ms/step - loss: 0.1374 - accuracy: 0.9583 - val_loss: 0.1557 - val_accuracy: 0.9450
Epoch 6/50
75/75 [==============================] - 1s 12ms/step - loss: 0.1645 - accuracy: 0.9483 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 7/50
75/75 [===================

### 4.3 Métricas de desempenho do modelo (2 pontos)

In [22]:
# Nome do arquivo onde o modelo treinado será salvo
model_teste = "liveness.keras"

print("[INFO] evaluating network...")

# Faz previsões sobre os dados de teste
predictions = model.predict(x=testX, batch_size=BS)

# Imprime um relatório de classificação com as métricas de avaliação
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=le.classes_))

# Salva o modelo treinado no disco
print("[INFO] serializing network to '{}'...".format(model_teste))
model.save(model_teste)

# Salva o codificador de rótulos no disco
f = open("le.pickle", "wb")
f.write(pickle.dumps(le))
f.close()

[INFO] evaluating network...
25/25 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

       Falso       0.96      1.00      0.98        98
        Real       1.00      0.96      0.98       102

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200

[INFO] serializing network to 'liveness.keras'...


## 5 Teste Fim-a-Fim

Simule a operação fim-a-fim, com uma imagem de entrada forjada (foto de foto de um rosto) e outra com uma imagem de rosto, exibindo o resultado da classificação e a pontuação de cada classe.

In [18]:
# Carrega o detector de faces
print("[INFO] loading face detector...")
net = cv2.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")

# Carrega o modelo de detecção de autenticidade de faces
print("[INFO] loading liveness detector...")
model = load_model("liveness.keras")
le = pickle.loads(open("le.pickle", "rb").read())

# Inicia a transmissão de vídeo da webcam
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)  # Espera 2 segundos para a câmera inicializar

while True:
    # Lê um frame da transmissão de vídeo
    frame = vs.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    
    # Prepara o frame para detecção de rostos
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # Loop sobre as detecções de rostos
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filtra detecções fracas
        if confidence > 0.50:
            # Calcula as coordenadas da caixa delimitadora do rosto
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(w, endX)
            endY = min(h, endY)
            
            # Extrai o rosto da imagem
            face = frame[startY:endY, startX:endX]
            face = cv2.resize(face, (32, 32))
            face = face.astype("float") / 255.0
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)
            
            # Faz a previsão de autenticidade do rosto
            preds = model.predict(face)[0]
            j = np.argmax(preds)
            label = le.classes_[j]

            # Se o rosto for considerado "Real", desenha uma caixa azul, senão, desenha uma caixa vermelha
            label = "{}: {:.4f}".format(label, preds[j])
            if label == "Real":
                cv2.putText(frame, label, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                              (255, 0, 0), 2)
            else:
                cv2.putText(frame, label, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                              (0, 0, 255), 2)

    # Mostra o frame com as caixas delimitadoras e rótulos
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # Se a tecla 'q' for pressionada, sai do loop
    if key == ord("q"):
        break

# Limpa as janelas e para a transmissão de vídeo
cv2.destroyAllWindows()
vs.stop()

[INFO] loading face detector...
[INFO] loading liveness detector...
[INFO] starting video stream...
1/1 [==============================] - 0s 23ms/step


>Com a implementação da solução na forma de uma aplicação do [Streamlit](https://www.streamlit.io/) (veja a pata streamlit-app e use o template) vale 1 ponto adicional.

**Pergunta**: Se utilizou o Streamlit, compartilhe a URL do aplicativo publicado:

**Resposta**:

## 6 Conclusões (2,5 pontos)

**Pergunta**: Dado todo o estudo e pesquisa, quais foram as conclusões sobre a solução, o que funcionou, o que não funcionou e quais os detalhes que observariam numa nova versão e melhorias do processo?

**Resposta**: